In [1]:
import os
os.chdir('../py')

import pandas as pd
import numpy as np
import feather

from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["patch.force_edgecolor"] = False
plt.rcParams['font.family'] = 'Ricty Diminised'
sns.set(style="whitegrid", font='Ricty Diminised', palette="muted", color_codes=True, rc={'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

import japanize_matplotlib

from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import LabelEncoder

# my modules
import EDA
import utils

from scipy.stats import ks_2samp

In [15]:
import gc

In [89]:
train = feather.read_dataframe('../input/feather/train.ftr')
test = feather.read_dataframe('../input/feather/test.ftr')

In [90]:
# =============================================================================
# load feature
# =============================================================================
FEATURE_DIR = '../feature/'
USE_FEATURE = ['f202']
if len(USE_FEATURE) > 0:
    tr_files = []
    te_files = []
    for f in USE_FEATURE:
        tr_file   = glob(f'../feature/{f}*tr*.ftr')
        tr_files += tr_file

        te_file   = glob(f'../feature/{f}*te*.ftr')
        te_files += te_file
else:
    tr_feature_path = '../feature/*tr*.ftr'
    te_feature_path = '../feature/*te*.ftr'

    tr_files = sorted(glob(tr_feature_path))
    te_files = sorted(glob(te_feature_path))

train = pd.concat([train, *[feather.read_dataframe(f) for f in tr_files]], axis=1)
test  = pd.concat([test, *[feather.read_dataframe(f) for f in te_files]], axis=1)

In [91]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,f202__Transaction_hour,f202__year,f202__month,f202__dow,f202__quarter,f202__hour,f202__day,f202__all_group_nan_sum,f202__all_group_0_count,f202__date_block
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,0.0,2017,12,5,4,0,2,0.534247,0.524510,201712
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,0.0,2017,12,5,4,0,2,0.525114,0.634615,201712
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,0.0,2017,12,5,4,0,2,0.481735,0.546256,201712
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,0.0,2017,12,5,4,0,2,0.518265,0.436019,201712
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,2017,12,5,4,0,2,0.312785,0.568106,201712


In [92]:
use_cols = ['TransactionAmt', 'dist1', 'dist2', 'f202__date_block', 'card1']

drop_cols = [col for  col in test.columns if col not in use_cols]

train.drop(columns=(drop_cols + ['isFraud']), inplace=True)

test.drop(columns=drop_cols, inplace=True)

In [26]:
import super_aggre

In [96]:
df = pd.concat([train, test], axis=0)

In [68]:
df['f202__date_block'].unique()

array([201712, 201801, 201802, 201803, 201804, 201805, 201806, 201807,
       201808, 201809, 201810, 201811, 201812])

In [97]:
df['date_block_num'] = df['f202__date_block'].map(dict(zip(df['f202__date_block'].unique(), range(df['f202__date_block'].nunique()))))

In [74]:
agg = df.groupby('date_block_num', as_index=False).agg({'TransactionAmt': ['min', 'mean', 'max', 'std']})

In [102]:
card_count = pd.crosstab(index=df['card1'],
                         columns=df['date_block_num'])

In [105]:
df

,TransactionAmt,card1,dist1,dist2,f202__date_block,date_block_num
0,68.500000,13926,19.0,NaN,201712,0
1,29.000000,2755,NaN,NaN,201712,0
2,59.000000,4663,287.0,NaN,201712,0
3,50.000000,18132,NaN,NaN,201712,0
4,50.000000,4497,NaN,NaN,201712,0
5,49.000000,5937,36.0,NaN,201712,0
6,159.000000,12308,0.0,NaN,201712,0
7,422.500000,12695,NaN,NaN,201712,0
8,15.000000,2803,NaN,NaN,201712,0
9,117.000000,17399,19.0,NaN,201712,0


In [106]:
df['card1'].map(card_count[0])

0            7
1          167
2          178
3          813
4            2
5            3
6           39
7         1407
8         1243
9          332
10           2
11         759
12          39
13         175
14         408
15           6
16           9
17          18
18         178
19         179
20          11
21          17
22         739
23         206
24         592
25         128
26         139
27           3
28         100
29         198
          ... 
506661       9
506662       0
506663     175
506664       2
506665     759
506666      75
506667       9
506668     332
506669     345
506670     759
506671     457
506672     345
506673       9
506674     453
506675     115
506676      31
506677    2633
506678      49
506679       8
506680     700
506681     700
506682       9
506683    1340
506684    1143
506685     214
506686     479
506687    1143
506688     195
506689       0
506690      40
Name: card1, Length: 1097231, dtype: int64

In [100]:
df.groupby('date_block_num').agg({'card1': 'count'})

,card1
date_block_num,
0,134339
1,92510
2,85725
3,101968
4,83571
5,89673
6,2754
7,75731
8,76252


In [87]:
agg = super_aggre.auto_agg(
    data=df,
    group='date_block_num',
    agg_cols=['TransactionAmt', 'dist1', 'dist2'],
    agg_funcs=['min', 'mean', 'max', 'std']
)

In [113]:
agg.shift(1).add_suffix('__lag1').reset_index()

,date_block_num,TransactionAmt_min__lag1,TransactionAmt_mean__lag1,TransactionAmt_max__lag1,TransactionAmt_std__lag1,dist1_min__lag1,dist1_mean__lag1,dist1_max__lag1,dist1_std__lag1,dist2_min__lag1,dist2_mean__lag1,dist2_max__lag1,dist2_std__lag1
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.292,128.279709,5094.950195,201.231628,0.0,124.842827,7068.0,380.586670,0.0,234.419449,9103.0,544.046753
2,2,0.292,133.950607,5278.950195,228.731796,0.0,134.430771,6818.0,405.398346,0.0,258.679382,6915.0,559.944946
3,3,0.498,138.767548,31937.390625,290.486694,0.0,141.379684,10286.0,410.283539,0.0,240.870209,8787.0,491.294006
4,4,0.251,140.420670,5543.229980,243.163132,0.0,99.167877,4806.0,325.215851,0.0,228.248657,11623.0,582.136475
5,5,0.364,133.698914,5279.950195,233.949982,0.0,100.668991,6466.0,332.466095,0.0,211.234558,10237.0,545.827026
6,6,0.499,136.656464,5366.819824,244.754150,0.0,109.675194,8081.0,365.461487,0.0,199.475601,7179.0,446.167816
7,7,1.001,171.459625,3163.939941,316.236938,0.0,116.683731,2751.0,351.194336,0.0,117.798447,2296.0,269.476959
8,8,0.246,141.522980,9154.459961,255.969604,0.0,93.183929,8081.0,332.536743,0.0,229.435333,8169.0,547.233032
9,9,0.018,140.425232,6759.000000,254.617096,0.0,87.548027,6466.0,316.930267,0.0,215.150955,8145.0,497.414520


In [115]:
test.index

RangeIndex(start=0, stop=506691, step=1)

In [119]:
df.iloc[:len(train)]

,TransactionAmt,card1,dist1,dist2,f202__date_block,date_block_num
0,68.500000,13926,19.0,NaN,201712,0
1,29.000000,2755,NaN,NaN,201712,0
2,59.000000,4663,287.0,NaN,201712,0
3,50.000000,18132,NaN,NaN,201712,0
4,50.000000,4497,NaN,NaN,201712,0
5,49.000000,5937,36.0,NaN,201712,0
6,159.000000,12308,0.0,NaN,201712,0
7,422.500000,12695,NaN,NaN,201712,0
8,15.000000,2803,NaN,NaN,201712,0
9,117.000000,17399,19.0,NaN,201712,0


In [120]:
df.iloc[len(train):]

,TransactionAmt,card1,dist1,dist2,f202__date_block,date_block_num
0,31.950001,10409,1.0,NaN,201807,7
1,49.000000,4272,4.0,NaN,201807,7
2,171.000000,4476,2635.0,NaN,201807,7
3,284.950012,10989,17.0,NaN,201807,7
4,67.949997,18018,6.0,NaN,201807,7
5,57.950001,12839,NaN,NaN,201807,7
6,87.000000,16560,NaN,NaN,201807,7
7,390.000000,15066,303.0,NaN,201807,7
8,103.949997,2803,3.0,NaN,201807,7
9,117.000000,12544,8.0,NaN,201807,7


In [109]:
df.merge(agg.shift(1).reset_index(), on='date_block_num', how='left')

,TransactionAmt,card1,dist1,dist2,f202__date_block,date_block_num,TransactionAmt_min,TransactionAmt_mean,TransactionAmt_max,TransactionAmt_std,dist1_min,dist1_mean,dist1_max,dist1_std,dist2_min,dist2_mean,dist2_max,dist2_std
0,68.500000,13926,19.0,NaN,201712,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29.000000,2755,NaN,NaN,201712,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59.000000,4663,287.0,NaN,201712,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50.000000,18132,NaN,NaN,201712,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,50.000000,4497,NaN,NaN,201712,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,49.000000,5937,36.0,NaN,201712,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,159.000000,12308,0.0,NaN,201712,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,422.500000,12695,NaN,NaN,201712,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,15.000000,2803,NaN,NaN,201712,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,117.000000,17399,19.0,NaN,201712,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
agg

date_block_num TransactionAmt                                      
                             min        mean           max         std
0               0          0.292  128.279709   5094.950195  201.231628
1               1          0.292  133.950607   5278.950195  228.731796
2               2          0.498  138.767548  31937.390625  290.486694
3               3          0.251  140.420670   5543.229980  243.163132
4               4          0.364  133.698914   5279.950195  233.949982
5               5          0.499  136.656464   5366.819824  244.754150
6               6          1.001  171.459625   3163.939941  316.236938
7               7          0.246  141.522980   9154.459961  255.969604
8               8          0.018  140.425232   6759.000000  254.617096
9               9          0.590  137.853455   6532.950195  244.831406
10             10          0.615  127.683777  10270.000000  255.761780
11             11          0.849  136.095917   6222.000000  250.513763
12             12          0.577  128.697311   7926.419922  222.717026

In [76]:
agg_lag1 = agg.copy()
agg_lag1['date_block_num'] = agg_lag1['date_block_num'] + 1

In [ ]:
def make_lag(df, lag):
    agg = df.groupby('date_block_num', as_index=False).agg({'Transaction'})

In [77]:
agg_lag1

date_block_num TransactionAmt                                      
                             min        mean           max         std
0               1          0.292  128.279709   5094.950195  201.231628
1               2          0.292  133.950607   5278.950195  228.731796
2               3          0.498  138.767548  31937.390625  290.486694
3               4          0.251  140.420670   5543.229980  243.163132
4               5          0.364  133.698914   5279.950195  233.949982
5               6          0.499  136.656464   5366.819824  244.754150
6               7          1.001  171.459625   3163.939941  316.236938
7               8          0.246  141.522980   9154.459961  255.969604
8               9          0.018  140.425232   6759.000000  254.617096
9              10          0.590  137.853455   6532.950195  244.831406
10             11          0.615  127.683777  10270.000000  255.761780
11             12          0.849  136.095917   6222.000000  250.513763
12             13          0.577  128.697311   7926.419922  222.717026

In [44]:
agg = df.groupby('f202__date_block', as_index=False).agg({'TransactionAmt': 'mean'})

In [48]:
agg_lag1 = agg.copy()
agg_lag1['f202_date_block'] = agg_lag1['f202__date_block'] - 1

In [49]:
agg

,f202__date_block,TransactionAmt
0,201712,128.279709
1,201801,133.950607
2,201802,138.767548
3,201803,140.420670
4,201804,133.698914
5,201805,136.656464
6,201806,171.459625
7,201807,141.522980
8,201808,140.425232
9,201809,137.853455


In [50]:
agg_lag1

,f202__date_block,TransactionAmt,f202_date_block
0,201712,128.279709,201711
1,201801,133.950607,201800
2,201802,138.767548,201801
3,201803,140.420670,201802
4,201804,133.698914,201803
5,201805,136.656464,201804
6,201806,171.459625,201805
7,201807,141.522980,201806
8,201808,140.425232,201807
9,201809,137.853455,201808


In [43]:
df.merge(agg_lag1, on='f202__date_block', how='left')

,TransactionAmt,dist1,dist2,f202__date_block,date_block_lag1,date_block_lag2,date_block_lag3,date_block_lag6
0,68.500000,19.0,NaN,201712,201711,201710,201709,201706
1,29.000000,NaN,NaN,201712,201711,201710,201709,201706
2,59.000000,287.0,NaN,201712,201711,201710,201709,201706
3,50.000000,NaN,NaN,201712,201711,201710,201709,201706
4,50.000000,NaN,NaN,201712,201711,201710,201709,201706
5,49.000000,36.0,NaN,201712,201711,201710,201709,201706
6,159.000000,0.0,NaN,201712,201711,201710,201709,201706
7,422.500000,NaN,NaN,201712,201711,201710,201709,201706
8,15.000000,NaN,NaN,201712,201711,201710,201709,201706
9,117.000000,19.0,NaN,201712,201711,201710,201709,201706


In [32]:
df.head()

,TransactionAmt,dist1,dist2,f202__date_block,date_block_lag1,date_block_lag2,date_block_lag3,date_block_lag6
0,68.5,19.0,NaN,201712,201711,201710,201709,201706
1,29.0,NaN,NaN,201712,201711,201710,201709,201706
2,59.0,287.0,NaN,201712,201711,201710,201709,201706
3,50.0,NaN,NaN,201712,201711,201710,201709,201706
4,50.0,NaN,NaN,201712,201711,201710,201709,201706


In [41]:
df.head()

,TransactionAmt,dist1,dist2,f202__date_block,date_block_lag1,date_block_lag2,date_block_lag3,date_block_lag6
0,68.5,19.0,NaN,201712,201711,201710,201709,201706
1,29.0,NaN,NaN,201712,201711,201710,201709,201706
2,59.0,287.0,NaN,201712,201711,201710,201709,201706
3,50.0,NaN,NaN,201712,201711,201710,201709,201706
4,50.0,NaN,NaN,201712,201711,201710,201709,201706


In [37]:
super_aggre.auto_agg(
    data=df,
    orig=df,
    agg_cols=['TransactionAmt', 'dist1', 'dist2'],
    group=['date_block_lag1'],
    agg_funcs=['min', 'mean', 'max', 'std']
)

,date_block_lag1,TransactionAmt_min,TransactionAmt_mean,TransactionAmt_max,TransactionAmt_std,dist1_min,dist1_mean,dist1_max,dist1_std,dist2_min,dist2_mean,dist2_max,dist2_std
0,201711,0.292,128.279709,5094.950195,201.231628,0.0,124.842827,7068.0,380.586670,0.0,234.419449,9103.0,544.046753
1,201711,0.292,128.279709,5094.950195,201.231628,0.0,124.842827,7068.0,380.586670,0.0,234.419449,9103.0,544.046753
2,201711,0.292,128.279709,5094.950195,201.231628,0.0,124.842827,7068.0,380.586670,0.0,234.419449,9103.0,544.046753
3,201711,0.292,128.279709,5094.950195,201.231628,0.0,124.842827,7068.0,380.586670,0.0,234.419449,9103.0,544.046753
4,201711,0.292,128.279709,5094.950195,201.231628,0.0,124.842827,7068.0,380.586670,0.0,234.419449,9103.0,544.046753
5,201711,0.292,128.279709,5094.950195,201.231628,0.0,124.842827,7068.0,380.586670,0.0,234.419449,9103.0,544.046753
6,201711,0.292,128.279709,5094.950195,201.231628,0.0,124.842827,7068.0,380.586670,0.0,234.419449,9103.0,544.046753
7,201711,0.292,128.279709,5094.950195,201.231628,0.0,124.842827,7068.0,380.586670,0.0,234.419449,9103.0,544.046753
8,201711,0.292,128.279709,5094.950195,201.231628,0.0,124.842827,7068.0,380.586670,0.0,234.419449,9103.0,544.046753
9,201711,0.292,128.279709,5094.950195,201.231628,0.0,124.842827,7068.0,380.586670,0.0,234.419449,9103.0,544.046753


In [121]:
def lag_merger(df, agg, lag):
    shift = agg.shift(lag).add_prefix(f'__lag{lag}').reset_index()
    df = df.merge(shift, on='date_block_num', how='left')

    return df

lags = [1, 2, 3, 6]
for lag in lags:
    df = lag_merger(df, agg, lag)

In [123]:
df.columns

Index(['TransactionAmt', 'card1', 'dist1', 'dist2', 'f202__date_block',
       'date_block_num', '__lag1TransactionAmt_min',
       '__lag1TransactionAmt_mean', '__lag1TransactionAmt_max',
       '__lag1TransactionAmt_std', '__lag1dist1_min', '__lag1dist1_mean',
       '__lag1dist1_max', '__lag1dist1_std', '__lag1dist2_min',
       '__lag1dist2_mean', '__lag1dist2_max', '__lag1dist2_std',
       '__lag2TransactionAmt_min', '__lag2TransactionAmt_mean',
       '__lag2TransactionAmt_max', '__lag2TransactionAmt_std',
       '__lag2dist1_min', '__lag2dist1_mean', '__lag2dist1_max',
       '__lag2dist1_std', '__lag2dist2_min', '__lag2dist2_mean',
       '__lag2dist2_max', '__lag2dist2_std', '__lag3TransactionAmt_min',
       '__lag3TransactionAmt_mean', '__lag3TransactionAmt_max',
       '__lag3TransactionAmt_std', '__lag3dist1_min', '__lag3dist1_mean',
       '__lag3dist1_max', '__lag3dist1_std', '__lag3dist2_min',
       '__lag3dist2_mean', '__lag3dist2_max', '__lag3dist2_std',
       '__l

In [122]:
df

,TransactionAmt,card1,dist1,dist2,f202__date_block,date_block_num,__lag1TransactionAmt_min,__lag1TransactionAmt_mean,__lag1TransactionAmt_max,__lag1TransactionAmt_std,...,__lag6TransactionAmt_max,__lag6TransactionAmt_std,__lag6dist1_min,__lag6dist1_mean,__lag6dist1_max,__lag6dist1_std,__lag6dist2_min,__lag6dist2_mean,__lag6dist2_max,__lag6dist2_std
0,68.500000,13926,19.0,NaN,201712,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29.000000,2755,NaN,NaN,201712,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59.000000,4663,287.0,NaN,201712,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50.000000,18132,NaN,NaN,201712,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,50.000000,4497,NaN,NaN,201712,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,49.000000,5937,36.0,NaN,201712,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,159.000000,12308,0.0,NaN,201712,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,422.500000,12695,NaN,NaN,201712,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,15.000000,2803,NaN,NaN,201712,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,117.000000,17399,19.0,NaN,201712,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
